<a href="https://colab.research.google.com/github/Levan-Danelia/FRTB/blob/main/FRTB_CRDL_Non_Securitization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Cell 1: Setup and Initial Data
# Import necessary libraries
import pandas as pd
import numpy as np

# --- Initial Portfolio Data ---
# This data represents the starting positions and their sensitivities.
data = [
    {'Position': 1, 'Bucket': 6, 'Curve': 'CDS', 'Sector': 'Tech', 'Tenor': '6M', 'Issuer': 'Issuer 1', 'Sensitivity': 98671},
    {'Position': 2, 'Bucket': 6, 'Curve': 'CDS', 'Sector': 'Tech', 'Tenor': '6M', 'Issuer': 'Issuer 1', 'Sensitivity': 131514},
    {'Position': 3, 'Bucket': 6, 'Curve': 'CDS', 'Sector': 'Tech', 'Tenor': '6M', 'Issuer': 'Issuer 2', 'Sensitivity': 98699},
    {'Position': 4, 'Bucket': 6, 'Curve': 'CDS', 'Sector': 'Tech', 'Tenor': '6M', 'Issuer': 'Issuer 2', 'Sensitivity': 131550},
    {'Position': 5, 'Bucket': 3, 'Curve': 'Bond', 'Sector': 'Financials', 'Tenor': '6M', 'Issuer': 'Issuer 3', 'Sensitivity': -156386},
    {'Position': 6, 'Bucket': 3, 'Curve': 'CDS', 'Sector': 'Financials', 'Tenor': '6M', 'Issuer': 'Issuer 3', 'Sensitivity': -2048},
    {'Position': 7, 'Bucket': 3, 'Curve': 'CDS', 'Sector': 'Financials', 'Tenor': '6M', 'Issuer': 'Issuer 3', 'Sensitivity': 1413}
]

# --- Regulatory Parameters ---
# Risk Weights for CSR Non-Securitisation Buckets as per Article 325ah
risk_weights = {
    3: 0.05,  # Financials, Investment Grade
    6: 0.02   # Technology, Investment Grade
}

# Cross-Bucket Correlation for CSR as per Article 325aj
# Correlation between Bucket 3 (Financials, IG) and Bucket 6 (Tech, IG)
gamma_bc_medium = 0.20

# Create an initial DataFrame
portfolio_df = pd.DataFrame(data)

print("--- Initial Setup Complete ---")
print(f"Loaded {len(portfolio_df)} positions.")

--- Initial Setup Complete ---
Loaded 7 positions.


In [2]:

# Cell 2: Step 1 & 2 - Identify Risk Factors and Individual Sensitivities
# The risk factor is a combination of Issuer, Tenor, and Curve.
# This step shows the initial portfolio with the sensitivity for each position.

print("### Step 1 & 2: Identify Risk Factors and Individual Sensitivities ###")
print("The initial portfolio with sensitivities for each position:")

# Display the initial portfolio DataFrame
display(portfolio_df.set_index('Position'))

### Step 1 & 2: Identify Risk Factors and Individual Sensitivities ###
The initial portfolio with sensitivities for each position:


,Bucket,Curve,Sector,Tenor,Issuer,Sensitivity
Position,,,,,,
1,6,CDS,Tech,6M,Issuer 1,98671
2,6,CDS,Tech,6M,Issuer 1,131514
3,6,CDS,Tech,6M,Issuer 2,98699
4,6,CDS,Tech,6M,Issuer 2,131550
5,3,Bond,Financials,6M,Issuer 3,-156386
6,3,CDS,Financials,6M,Issuer 3,-2048
7,3,CDS,Financials,6M,Issuer 3,1413


In [3]:

# Cell 3: Step 3 - Calculate Net Sensitivities (sk)
# As per Article 325f(5), we net sensitivities for identical risk factors.

print("\n### Step 3: Calculate Net Sensitivities (sk) ###")
print("Sensitivities are grouped by their unique risk factor and summed.")

# Define the risk factor components
risk_factor_cols = ['Issuer', 'Tenor', 'Curve', 'Bucket', 'Sector']

# Group by risk factors and sum sensitivities to get the net sensitivity
net_sensitivities_df = portfolio_df.groupby(risk_factor_cols).agg(
    Net_Sensitivity_sk=('Sensitivity', 'sum')
).reset_index()

display(net_sensitivities_df)


### Step 3: Calculate Net Sensitivities (sk) ###
Sensitivities are grouped by their unique risk factor and summed.


,Issuer,Tenor,Curve,Bucket,Sector,Net_Sensitivity_sk
0,Issuer 1,6M,CDS,6,Tech,230185
1,Issuer 2,6M,CDS,6,Tech,230249
2,Issuer 3,6M,Bond,3,Financials,-156386
3,Issuer 3,6M,CDS,3,Financials,-635


In [4]:
# Cell 4: Step 4 - Calculate Weighted Sensitivities (WSk)
# Each net sensitivity is multiplied by its regulatory risk weight (RWk) from Article 325ah.

print("\n### Step 4: Calculate Weighted Sensitivities (WSk) ###")
print("Applying regulatory risk weights to the net sensitivities.")

# Map the risk weights to the DataFrame
net_sensitivities_df['Risk_Weight_RWk'] = net_sensitivities_df['Bucket'].map(risk_weights)

# Calculate the weighted sensitivity
net_sensitivities_df['Weighted_Sensitivity_WSk'] = net_sensitivities_df['Net_Sensitivity_sk'] * net_sensitivities_df['Risk_Weight_RWk']

display(net_sensitivities_df[['Bucket', 'Sector', 'Issuer', 'Tenor', 'Curve', 'Net_Sensitivity_sk', 'Risk_Weight_RWk', 'Weighted_Sensitivity_WSk']])


### Step 4: Calculate Weighted Sensitivities (WSk) ###
Applying regulatory risk weights to the net sensitivities.


,Bucket,Sector,Issuer,Tenor,Curve,Net_Sensitivity_sk,Risk_Weight_RWk,Weighted_Sensitivity_WSk
0,6,Tech,Issuer 1,6M,CDS,230185,0.02,4603.70
1,6,Tech,Issuer 2,6M,CDS,230249,0.02,4604.98
2,3,Financials,Issuer 3,6M,Bond,-156386,0.05,-7819.30
3,3,Financials,Issuer 3,6M,CDS,-635,0.05,-31.75


In [5]:
# Cell 5: Step 5, 6 & 7 - Intra-Bucket Aggregation (Kb)
# This function calculates the bucket-specific capital (Kb) by aggregating
# the weighted sensitivities within each bucket using the intra-bucket correlation.

def calculate_kb(bucket_df, scenario='medium'):
    """
    Calculates the bucket-specific capital (Kb) based on Article 325f(7).
    """
    ws_vector = bucket_df[['Issuer', 'Tenor', 'Curve', 'Weighted_Sensitivity_WSk']].values

    # Sum of squares of weighted sensitivities
    sum_ws_sq = np.sum(ws_vector[:, 3]**2)

    # Sum of cross-products
    cross_product_sum = 0
    if len(ws_vector) > 1:
        for i in range(len(ws_vector)):
            for j in range(len(ws_vector)):
                if i != j:
                    # Determine intra-bucket correlation (rho_kl) as per Article 325ai
                    rho_name = 1.0 if ws_vector[i, 0] == ws_vector[j, 0] else 0.35 # Same vs Different Issuer (IG)
                    rho_tenor = 1.0 if ws_vector[i, 1] == ws_vector[j, 1] else 0.999 # Same vs Different Tenor
                    rho_basis = 1.0 if ws_vector[i, 2] == ws_vector[j, 2] else 0.999 # Same vs Different Basis

                    rho_kl_medium = rho_name * rho_tenor * rho_basis

                    # Adjust correlation based on the scenario as per Article 325h
                    if scenario == 'high':
                        rho_kl = min(rho_kl_medium * 1.25, 1.0)
                    elif scenario == 'low':
                        rho_kl = max(2 * rho_kl_medium - 1, 0.75 * rho_kl_medium)
                    else: # medium
                        rho_kl = rho_kl_medium

                    cross_product_sum += rho_kl * ws_vector[i, 3] * ws_vector[j, 3]

    k_squared = max(sum_ws_sq + cross_product_sum, 0)
    return np.sqrt(k_squared)

print("\n### Step 5, 6 & 7: Intra-Bucket Aggregation (Kb) ###")
print("Calculating bucket-specific capital (Kb) for the Medium Correlation Scenario.")

# Calculate Kb and Sb for each bucket for the medium scenario
bucket_capital_medium = {}
for bucket_id, group in net_sensitivities_df.groupby('Bucket'):
    bucket_capital_medium[bucket_id] = calculate_kb(group, scenario='medium')

s_b_values = net_sensitivities_df.groupby('Bucket')['Weighted_Sensitivity_WSk'].sum().to_dict()

kb_results_df = pd.DataFrame({
    'Bucket': list(bucket_capital_medium.keys()),
    'Kb_Medium': list(bucket_capital_medium.values()),
    'Sb_Medium': list(s_b_values.values())
})

display(kb_results_df)


### Step 5, 6 & 7: Intra-Bucket Aggregation (Kb) ###
Calculating bucket-specific capital (Kb) for the Medium Correlation Scenario.


,Bucket,Kb_Medium,Sb_Medium
0,3,7851.018378,-7851.05
1,6,7565.702676,9208.68


In [6]:
# Cell 6: Step 8 - Across-Bucket Aggregation (Medium Scenario)
# This function aggregates the bucket-level capital (Kb) to get the final capital for a given scenario.

def calculate_final_capital(kb_df, scenario='medium'):
    """
    Calculates the final risk class capital based on Article 325f(8).
    """
    # Adjust correlation based on the scenario as per Article 325h
    if scenario == 'high':
        gamma = min(gamma_bc_medium * 1.25, 1.0)
    elif scenario == 'low':
        gamma = max(2 * gamma_bc_medium - 1, 0.75 * gamma_bc_medium)
    else: # medium
        gamma = gamma_bc_medium

    k_values = kb_df[f'Kb_{scenario.capitalize()}'].values
    s_values = kb_df[f'Sb_{scenario.capitalize()}'].values

    sum_k_sq = np.sum(k_values**2)

    cross_bucket_sum = 0
    if len(k_values) > 1:
        # This is simplified for two buckets; a loop is needed for more than two.
        cross_bucket_sum = 2 * gamma * s_values[0] * s_values[1]

    final_capital_sq = max(sum_k_sq + cross_bucket_sum, 0)
    return np.sqrt(final_capital_sq)

print("\n### Step 8: Across-Bucket Aggregation (Medium Scenario) ###")
print("Aggregating bucket capital using cross-bucket correlations.")

medium_scenario_capital = calculate_final_capital(kb_results_df, scenario='medium')
print(f"\nMedium Scenario CSR Delta Capital Requirement: {medium_scenario_capital:,.2f}")


### Step 8: Across-Bucket Aggregation (Medium Scenario) ###
Aggregating bucket capital using cross-bucket correlations.

Medium Scenario CSR Delta Capital Requirement: 9,484.68


In [7]:
# Cell 7: Step 9 - Calculate High and Low Correlation Scenarios
# Per Article 325h, the calculation is repeated for high and low correlation scenarios.

print("\n### Step 9: Calculate High and Low Correlation Scenarios ###")

# --- High Correlation Scenario ---
bucket_capital_high = {}
for bucket_id, group in net_sensitivities_df.groupby('Bucket'):
    bucket_capital_high[bucket_id] = calculate_kb(group, scenario='high')
kb_results_df['Kb_High'] = kb_results_df['Bucket'].map(bucket_capital_high)
kb_results_df['Sb_High'] = kb_results_df['Bucket'].map(s_b_values) # Sb values do not change
high_scenario_capital = calculate_final_capital(kb_results_df, scenario='high')

# --- Low Correlation Scenario ---
bucket_capital_low = {}
for bucket_id, group in net_sensitivities_df.groupby('Bucket'):
    bucket_capital_low[bucket_id] = calculate_kb(group, scenario='low')
kb_results_df['Kb_Low'] = kb_results_df['Bucket'].map(bucket_capital_low)
kb_results_df['Sb_Low'] = kb_results_df['Bucket'].map(s_b_values) # Sb values do not change
low_scenario_capital = calculate_final_capital(kb_results_df, scenario='low')

# --- Display Summary ---
scenario_summary = pd.DataFrame([
    {'Scenario': 'Medium', 'Capital': medium_scenario_capital},
    {'Scenario': 'High', 'Capital': high_scenario_capital},
    {'Scenario': 'Low', 'Capital': low_scenario_capital}
])
display(scenario_summary.set_index('Scenario'))


# Cell 8: Step 10 - Final Charge Calculation
# The final own funds requirement is the maximum of the three scenarios.

print("\n### Step 10: Final Charge Calculation ###")

final_capital_charge = scenario_summary['Capital'].max()

print(f"\nFinal CSR Delta Capital Requirement: {final_capital_charge:,.2f}")

summary_card = pd.DataFrame([{'Final CSR Delta Capital Requirement': f"£{final_capital_charge:,.2f}"}])
display(summary_card)


### Step 9: Calculate High and Low Correlation Scenarios ###


,Capital
Scenario,
Medium,9484.683638
High,9297.307683
Low,9668.428894



### Step 10: Final Charge Calculation ###

Final CSR Delta Capital Requirement: 9,668.43


,Final CSR Delta Capital Requirement
0,"£9,668.43"
